In [73]:
import logging
import re
import sys
import openai
from dotenv import load_dotenv, find_dotenv
from llama_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from llama_index import ( 
    SimpleWebPageReader, 
) 
from langchain.chat_models import ChatOpenAI
from langchain.llms  import AzureOpenAI
from llama_index import StorageContext, load_index_from_storage 
from langchain.embeddings import OpenAIEmbeddings
import os

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

_ = load_dotenv(find_dotenv('.env_x'),override=True)   


if(os.environ['OPENAI_API_TYPE'] == 'azure'):
            openai.api_type = os.environ['OPENAI_API_TYPE']
            openai.api_base = os.environ['OPENAI_API_BASE']
            openai.api_version = os.environ['OPENAI_API_VERSION']
            openai.api_key = os.environ['OPENAI_API_KEY']                     
else:
            openai.api_key = os.environ['OPENAI_API_KEY']
            openai.api_base = "https://api.openai.com/v1"
            openai.api_type = os.environ['OPENAI_API_TYPE']
            openai.api_version = "" 

dataSource = ["https://support.microsoft.com/en-us/surface/surface-pro-8-update-history-1080bf34-7e87-408c-8619-80571283526e",
              "https://support.microsoft.com/en-us/surface/surface-book-3-update-history-935a7b6b-2f6d-dbf9-b3ba-0ea61e187b2d"]

urls = dataSource
   
documents = SimpleWebPageReader(html_to_text=False).load_data(urls)   
#key_words = "Intel\(R\) Iris\(R\) Plus Graphics - Extn"
current_version = "16.0.100.9000"
key_words = "Surface UEFI [-\u2013] Firmware"
#key_words = "Surface Camera Front - System devices"

prompt = "The device driver "+key_words+" has these versions in the list: "
for document in documents:
    matches = re.findall(r'<p>(.*?)<\/p>[^<(.?)*]*<\/td>[^<(.?)*]*<td>[^<(.?)*]*<p>{0}<\/p>'.format(key_words), document.text, re.IGNORECASE)
    prompt = prompt + "\r\n" + '\r\n'.join(matches)

print(prompt)

prompt = prompt + "\r\n\r\n" + "The current version is: " + current_version + "\r\n" + " The version number format is Major.Minor.Patch.MinorPatch. \
    The latest version number in the list is the highest one. If the current version number is above the highest one, \
        then don't need to update. Based on this tell me if the current version should be updated or not. \
            If yes, please tell me the latest version."

messages = [{"role": "user", "content": prompt}]
response = get_completion_from_messages(messages)

print(response)





The device driver Surface UEFI [-–] Firmware has these versions in the list: 
Surface - Firmware - 18.104.143.0
Surface - Firmware - 18.102.141.0
Surface - Firmware - 6.210.141.0
Surface - Firmware - 6.209.141.0
Surface - Firmware - 13.101.140.0
Surface - Firmware - 13.101.140.0
Surface - Firmware - 10.105.140.0
Surface – Firmware – 10.104.140.0
Surface – Firmware – 10.101.140.0
Surface – Firmware – 9.102.140.0
Surface – Firmware – 9.101.140.0
The current version (16.0.100.9000) is not listed in the versions provided. The latest version in the list is Surface - Firmware - 18.104.143.0. Therefore, the current version should be updated to the latest version.
